순환 신경망 모델은 순차적인 자료에서 규칙적인 패턴을 인식하거나 그 의미를 추론할 수 있다.<br>
순차적이라는 특성 때문에 간단한 레이어로도 다양한 형태의 모델을 구성할 수 있다.<br>
케라스에서 제공하는 순환 신경망 레이어 : SimpleRNN, GRU, LSTM이 있으나 주로 LSTM을 사용한다.

# 1. 긴 시퀀스를 기억할 수 있는 LSTM(Long Short-Term Memory units) 레이어
LSTM 레이어는 아래와 같이 간단히 사용할 수 있다.<br>
- 입력 형태
 
    LSTM(3, input_dim=1)
  
기본 인자 : <br>
1. 1번째 인자 : 메모리 셀의 개수
2. 2번째 인자 : 입력 속성 수

Dense 레이어 형태와 비슷하다.<br>
첫번째 인자인 메모리 셀의 개수는 기억용량 정도와 출력 형태를 결정짓는다. = Dense 레이어에서의 출력 뉴런 수와 비슷<br>
input_dim은 Dense 레이어와 같이 일반적으로 속성의 개수가 들어간다.<br>
    
    Dense(3, input_dim=1)
    
LSTM의 한 가지 인자에 대해 더 알아보겠다.<br>
 
    LSTM(3, input_dim=1, input_length=4)
    
- input_length : 시퀀스 데이터의 입력 길이

Dense와 LSTM 도식화 참고(p.146)<br>
LSTM는 Dense와 비교하면 뉴련들이 튀어나와 있다.<br>
input_length가 길다고 해서 각 입력마다 다른 가중치가 사용하는 것이 입력 길이 만큼 연결한 것이기 때문에 모두 동일한 가중치를 공유함.<br>

- 출력 형태 : return_sequences : 시퀀스 출력 여부

LSTM 레이어는 return_sequences 인자에 따라 마지막 시퀀스에서 한 번만 출력할 수 있고 가가 시퀀스에서 출력할 수 있다.<br>
Many to many 문제를 풀거나 LSTM 레이어를 여러 개로 쌓아 올릴 때는 return_sequences=True 옵션을 사용한다.<br>
return_sequences=True -> 출력값이 여러 개, return_sequences=False -> 출력값이 1개<br>

- 상태유지(stateful) 모드 : stateful : 상태 유지 여부

학습 샘플의 가장 마지막 상태가 다음 샘플 학습 시에 입력으로 전달 여부를 지정하는 것<br>
하나의 샘플은 4개의 시퀀스 입력이 있고 총 3개의 샘플 있을 때(p.148 참고)<br>
- stateful=True일 때 : 도출된 현재 상태의 가중치가 다음 샘플 학습 시의 초기 상태로 입력됨
- stateful=False일 때 : 도출된 현재 상태의 가중치가 다음 샘플 학습 시의 초기 상태로 입력되지 않음


# 2. 순환 신경망 모델 만들기

### 1. 시퀀스 데이터 준비
순환 신경망은 주로 자연어 처리에 많이 쓰인다. -> 문장 학습 예제가 일반적이다. 예제는 악보로 하겠다.<br>
악보 학습이 더 쉬운 이유는<br> 
- 음계가 문장보다 더 코드화하기 쉽다.
- 시계열 자료이다.
- 나온 결과를 악보로 볼 수 있다.
- 우리가 학습한 모델이 연주하는 곡을 들어볼 수 있기 때문이다.

여기서 예제는 나비야로 하겠다.

### 2. 데이터셋 생성